## Imports

In [1]:
import time, os, csv, json
from music21 import *
import dill as pickle
from multiprocessing import Pool as ThreadPool
import numpy as np
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector

## Extract Features

In [ ]:
def extract_features(midi_path):

    try:
        s = converter.parse(midi_path)
    except:
        return None
    
    # for a description of features, see section 5 of the jSymbolic paper
    # http://jmir.sourceforge.net/publications/MA_Thesis_2004_Bodhidharma.pdf
    feats = [
        # instrument features-----------------------------------
#         features.jSymbolic.PitchedInstrumentsPresentFeature,
#         features.jSymbolic.NotePrevalenceOfPitchedInstrumentsFeature,
        features.jSymbolic.NumberOfPitchedInstrumentsFeature,
        features.jSymbolic.StringKeyboardFractionFeature,
        features.jSymbolic.AcousticGuitarFractionFeature,
        features.jSymbolic.ElectricGuitarFractionFeature,
        features.jSymbolic.ViolinFractionFeature,
        features.jSymbolic.SaxophoneFractionFeature,
        features.jSymbolic.BrassFractionFeature,
        features.jSymbolic.WoodwindsFractionFeature,
        features.jSymbolic.OrchestralStringsFractionFeature,
        features.jSymbolic.StringEnsembleFractionFeature,
        features.jSymbolic.ElectricInstrumentFractionFeature,
        # rythm features----------------------------------------
        features.jSymbolic.InitialTempoFeature,
        features.jSymbolic.InitialTimeSignatureFeature,
        features.jSymbolic.NoteDensityFeature,
        features.jSymbolic.AverageNoteDurationFeature,
        features.jSymbolic.AverageTimeBetweenAttacksFeature,
        features.jSymbolic.VariabilityOfTimeBetweenAttacksFeature,
        features.jSymbolic.AverageTimeBetweenAttacksForEachVoiceFeature,
        features.jSymbolic.AverageVariabilityOfTimeBetweenAttacksForEachVoiceFeature,
        features.jSymbolic.ChangesOfMeterFeature,
        features.jSymbolic.MaximumNoteDurationFeature,
        features.jSymbolic.MinimumNoteDurationFeature,
        # pitch statistics features-----------------------------
        features.jSymbolic.MostCommonPitchPrevalenceFeature,
        features.jSymbolic.MostCommonPitchClassPrevalenceFeature,
        features.jSymbolic.PitchVarietyFeature,
        features.jSymbolic.PitchClassVarietyFeature,
        features.jSymbolic.RangeFeature,
        features.jSymbolic.ImportanceOfBassRegisterFeature,
        features.jSymbolic.ImportanceOfMiddleRegisterFeature,
        features.jSymbolic.ImportanceOfHighRegisterFeature,
        features.jSymbolic.MelodicIntervalHistogramFeature,
        features.jSymbolic.BasicPitchHistogramFeature,
        features.jSymbolic.PitchClassDistributionFeature,
        features.jSymbolic.QualityFeature, # mode
#         features.jSymbolic.GlissandoPrevalenceFeature,
#         features.jSymbolic.VibratoPrevalenceFeature,
        # melody features---------------------------------------
        features.jSymbolic.MelodicIntervalHistogramFeature,
        features.jSymbolic.AmountOfArpeggiationFeature,
        features.jSymbolic.RepeatedNotesFeature,
        features.jSymbolic.ChromaticMotionFeature,
        features.jSymbolic.StepwiseMotionFeature,
        features.jSymbolic.MelodicThirdsFeature,
        features.jSymbolic.MelodicFifthsFeature,
        features.jSymbolic.MelodicTritonesFeature,
        features.jSymbolic.MelodicOctavesFeature,
        features.jSymbolic.DirectionOfMotionFeature
    ]
    
    ds = features.DataSet(classLabel=midi_path)
    ds.addData(s)
    ds.addFeatureExtractors(feats)
    ds.process()
    return (ds.getFeaturesAsList(), ds.getAttributeLabels())
        
symlink_dir = '../../data/query_symlinks'

num_threads = 8
pool = ThreadPool(num_threads)

start_time = time.time()
extracted_features = pool.map(extract_features, [os.path.join(symlink_dir, n) for n in os.listdir(symlink_dir)])
print('Finished in {:.2f} seconds'.format(time.time() - start_time))

## Save/Load cached feature extraction

In [2]:
## save cache
# with open('1000_midi_files_features.pickle', 'w') as f:
#     pickle.dump(f, extracted_features)

## load cache
with open('1000_midi_files_features.pickle', 'r') as f:
    extracted_features = pickle.load(f)

## Load MSD Cache

In [3]:
start_time = time.time()
with open('../../data/msd.pickle', 'r') as f:
    msd = pickle.load(f)
print('Loaded in {:.2f} seconds'.format(time.time() - start_time))

Loaded in 10.58 seconds


## Analyze and remove unnecessary features

In [4]:
vec = []
for f in extracted_features:
    if f is not None:
        arr = f[0][0]
        arr.pop(0) # first element is an empty string
        arr.pop(-1) # last element is an empty string
        vec.append(np.array(arr))

In [5]:
embeddings = np.asarray(vec)
# summed = np.sum(embeddings, axis=0)
# for i, n in enumerate(results[0][1][1:-1]):
#     print('{}:     {}'.format(n, summed[i]))

## Tensorboard Labels/Metadata

In [6]:
LOG_DIR='../../data/embedding_logdir'
with open('/home/bbpwn2/Documents/code/midi-dataset/data/match_scores.json', 'r') as f:
    match_scores = json.load(f)

In [7]:
def get_mid_to_track(match_scores):
    mid_to_track = {}
    for k, vals in match_scores.items():
        for v in vals:
            if not v in mid_to_track:
                mid_to_track[v] = {
                    'track': k,
                    'confidence': match_scores[k][v]
                }
            # if the new confidence score is higher than the last
            # update the track it points to
            elif match_scores[k][v] > mid_to_track[v]['confidence']:
                mid_to_track[v]['track'] = k
    return mid_to_track

def get_track_to_msd(msd):
    return { m['track_id']: m for m in msd }

def save_features_to_tsv(filename, features, mid_to_track, track_to_msd):
    with open(filename, 'w') as csvfile:
        
        fieldnames = ['path', 
                      'song_year',
                      'song_title',
                      'song_time_signature',
                      'song_tempo',
                      'song_key',
                      'song_mode',
                      'song_loudness',
                      'song_energy',
                      'song_duration',
                      'song_danceability',
                      'song_hotttnesss',
                      'artist_name',
                      'artist_terms',
                      'artist_mbtags',
                      'artist_hotttnesss',
                      'artist_location']

        writer = csv.DictWriter(csvfile, delimiter='\t', fieldnames=fieldnames)

        writer.writeheader()
        for f in extracted_features:
            if f is not None:
                path = f[1][-1]
                basename = os.path.basename(path)[0:-4]
                if basename in mid_to_track:
                    track = mid_to_track[basename]['track']
                    d = {k: v for k, v in track_to_msd[track].items() if k in fieldnames}
                    for k, v in d.items():
                        if isinstance(v, np.ndarray):
                            d[k] = ','.join(v.tolist())
                    writer.writerow(d)                     

mid_to_track = get_mid_to_track(match_scores)
track_to_msd = get_track_to_msd(msd)
tsv = save_features_to_tsv(os.path.join(LOG_DIR, 'metadata.tsv'), extracted_features, mid_to_track, track_to_msd)

## Tensorboard Embeddings

In [8]:

sess = tf.Session()

# create embeddings var
emb = tf.Variable(embeddings, name='embeddings')

# embedding projector
summary_writer = tf.train.SummaryWriter(LOG_DIR)
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
embedding.tensor_name = emb.name
embedding.metadata_path = os.path.join(LOG_DIR, 'metadata.tsv')
projector.visualize_embeddings(summary_writer, config)

# init and run the session
init = tf.global_variables_initializer()
sess.run(init)

# save checkpoint
saver = tf.train.Saver()
saver.save(sess, os.path.join(LOG_DIR, "model.ckpt"), 0)

Instructions for updating:
Please switch to tf.summary.FileWriter. The interface and behavior is the same; this is just a rename.


'../../data/embedding_logdir/model.ckpt-0'

In [ ]:
# def histogram(vec):
#     hist = {}
#     for v in vec:
#         if not v in hist:
#             hist[v] = 0
#         hist[v] = hist[v] + 1
#     return [(k, v) for k, v in hist.items()]